Before running, install required packages:

In [ ]:
! pip install numpy sklearn tensorboardX

---

In [ ]:
import numpy as np
import sklearn
from sklearn.svm import SVC
from tensorboardX import SummaryWriter
from datetime import datetime

In [ ]:
def fake_data():
    # 4 images of shape 1x16x16 with labels 0, 1, 2, 3
    return [np.random.rand(4, 1, 16, 16), np.arange(4)]

# Setup

In [ ]:
# INSERT YOUR DATA HERE
# Expected format: [images, labels]
# - images has array shape (num samples, color channels, height, width)
# - labels has array shape (num samples, )
train_data = fake_data()  # required
val_data = fake_data()    # optional
test_data = None          # optional

In [ ]:
# Set up logging.
experiment_id = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
writer = SummaryWriter(logdir=f"logs/{experiment_id}")

# Preprocessing

In [ ]:
# Set up scaler.
scaler = sklearn.preprocessing.StandardScaler()

In [ ]:
def preprocess(data, name):
    if data is None:  # val/test can be empty
        return None
    images, labels = data

    # Flatten.
    images = images.reshape(len(images), -1)

    # Scale to mean 0 and std 1.
    if name == "train":
        scaler.fit(images)
    images = scaler.transform(images)

    # Shuffle train set.
    if name == "train":
        images, labels = sklearn.utils.shuffle(images, labels)

    return [images, labels]

In [ ]:
processed_train_data = preprocess(train_data, "train")
processed_val_data = preprocess(val_data, "val")
processed_test_data = preprocess(test_data, "test")

# Model

In [ ]:
model = SVC()

# Training

In [ ]:
def evaluate(data, name):
    if data is None:  # val/test can be empty
        return

    images, labels = data
    acc = model.score(images, labels)
    print(f"{name + ':':6} accuracy: {acc}")
    writer.add_scalar(f"{name}_accuracy", acc)

In [ ]:
# Train on train_data.
model.fit(*processed_train_data)

In [ ]:
# Evaluate on all datasets.
evaluate(processed_train_data, "train")
evaluate(processed_val_data, "val")
evaluate(processed_test_data, "test")